In [1]:
import numpy as np
from gensim.models import LsiModel
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity
from sklearn.metrics.pairwise import cosine_similarity
from utility.postgres_manager import *
import gensim
from ml_classes.mm_with_meta import *
corpus_path = "./gensim_data/updated_subjects/politic-ai-corpus.mm"
trained_path = "./gensim_data/updated_subjects/trained/politic-ai-trained.tr"
n_topics = 500
n_clusters = 100

postgresManager = PostgresManager()

D:\Programe\Anaconda3\envs\MachineLearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


connecting to PostgreSQL database...
connection established
PostgreSQL 10.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit


In [2]:
dictionary = gensim.corpora.Dictionary.load(corpus_path+".dict")
corpus = MmCorpusMeta(corpus_path, id2word=dictionary, metadata=True)
print(corpus)


MmCorpus(1054 documents, 23876 features, 526706 non-zero entries)


In [3]:
tfidf = TfidfModel(corpus, id2word=dictionary)

In [4]:
corpus_tfidf = tfidf[corpus]

In [20]:
from operator import itemgetter

def getTopKeywordsId(tfidf, doc, topn):
    keywords = tfidf[doc]              
    keywords.sort(key=itemgetter(1), reverse=True)
    return keywords[:topn]


In [28]:
def insertKeywordsIntoDatabase(postgresManager,subject_id,keywordsIds,dictionary):
    for keywordId in keywordsIds:
        word = dictionary[keywordId[0]]
        postgresManager.insert_with_args(
            "insert into politicalai_ict.keyword(id,subject_id,keyword)"
            " values(DEFAULT ,%s,%s)",
            (subject_id, word))

In [33]:
postgresManager.commit_changes()
for id,doc in corpus.iter_doc():
    keywords = getTopKeywordsId(tfidf,corpus[id],10)
    insertKeywordsIntoDatabase(postgresManager,corpus.doc_id_to_postgres_id[id],keywords,dictionary)
postgresManager.commit_changes()

In [17]:
getTopKeywordsId(tfidf,corpus[0],10)
dictionary[638]

'verkiezingen'

In [30]:
postgresManager.commit_changes()

In [31]:
keywords = getTopKeywordsId(tfidf,corpus[0],10)
insertKeywordsIntoDatabase(postgresManager,corpus.doc_id_to_postgres_id[0],keywords,dictionary)

In [32]:
postgresManager.commit_changes()